## 2. <a id='toc2_'></a>[Problem 2: Labor adjustment costs](#toc0_)

In [2]:
#impports and magics
import numpy as np
from scipy import linalg
from scipy import optimize
import sympy as sm

# local modules
import opg2py
from opg2py import ProfitClass
model = ProfitClass()

initializing the model
calling.setup()


You own a hair salon. You employ hairdressers, $\ell_t$, to produce haircuts, $y_t = \ell_t$.

The wage for each haridresser is $w$.

The demand for haircuts implies that the price of haircuts you can charge is $p_t = \kappa_t y_t^{-\eta}$, where $\kappa_t$ is a demand-shock and $\eta \in (0,1)$ measures the elasticity of demand.

Profits are:

$$
\Pi_t = p_t y_t - w \ell_t = \kappa_t \ell_t^{1-\eta} - w \ell_t
$$

Baseline parameters are:
- $\eta = 0.5$
- $w = 1.0$

**Question 1:** Verify numerically that $\ell_{t}=\left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}}$ maximises profits, for $\kappa\in\left\{1.0 , 2.0\right\}$.

As statet in the problemset the profits are given by:
$$
\Pi_t = p_t y_t - w \ell_t = \kappa_t \ell_t^{1-\eta} - w \ell_t
$$
This can be simplified for the following for $\kappa=1$
$$
\Pi_t=\ell_t^{0.5}-\ell_t
$$
And for $\kappa=2$
$$
\Pi_t=2\ell_t^{0.5}-\ell_t
$$

We see that when $\kappa=1$ we have the following value for $\ell_t$:

$$
\ell_t=\left(\frac{(1-0.5)*1}{1}\right)^{\frac{1}{0.5}}
\\
\ell_t=\left(\frac{0.5}{1}\right)^{\frac{1}{0.5}}=0.25
$$

We see that when $\kappa=2$ we have the following value for $\ell_t$:
$$
\ell_t=\left(\frac{(1-0.5)*2}{1}\right)^{\frac{1}{0.5}}
\\
\ell_t=\left(\frac{1}{1}\right)^{\frac{1}{0.5}}
\\
\ell_t=1^2=1
$$

We also make python solve this numiricly in the py.file by using sympy 

In [3]:
model = ProfitClass(do_print=False)
model.solve_numerical_kappa1()

Optimal ell_t value for kappa = 1.0: 0.250000000000000


In [4]:
model = ProfitClass(do_print=False)
model.solve_numerical_kappa2()

Optimal ell_t value for kappa = 2.0: 1.00000000000000


We now consider a *dynamic* version of the model.

* The demand-shock is a so-called AR(1) in logs, 

$$
\log \kappa_{t} = \rho \log \kappa_{t-1} + \epsilon_{t},\,\,\, \epsilon_{t+1} \sim \mathcal{N}(-0.5\sigma_{\epsilon}^2,\sigma_{\epsilon})
$$

* Any hiring or firing implies a fixed adjustment cost, $\iota > 0 $.
* Future profits are discounted with a monthly factor of $R \in (0,1)$.

The initial demand shock is $\kappa_{-1} = 1$ and the planning horizon is 10 years, i.e. 120 months so $t \in \{0,1,2,\dots,119\}$. Initially you don't have any employees, $\ell_{-1}=0$

The *ex post* value of the salon is *conditional* on the shock series is:

$$
h(\epsilon_0,\epsilon_1,\dots,\epsilon_{119}) = \left[\sum_{t=0}^{119}R^{-t}\left[\kappa_{t}\ell_{t}^{1-\eta}-w\ell_{t}-\boldsymbol{1}_{\ell_{t}\neq\ell_{t-1}}\iota\right]\right]
$$

The *ex ante* expected value of the salon can be approximated by

$$
H = \mathbb{E}[h(\epsilon_0,\epsilon_1,\dots,\epsilon_{119})] \approx \frac{1}{K}\sum_{k=0}^{K} h(\epsilon_0^k,\epsilon_1^k,\dots,\epsilon_{119}^k)
$$

where each $k\in\{0,1,\dots,K-1\}$ is a random shock series. Maximizing profitability means maximizing $H$.


Baseline parameters are: 

- $\rho = 0.90$
- $\iota = 0.01$
- $\sigma_{\epsilon} = 0.10$
- $R = \left(1+0.01\right)^{1/12}$

**Question 2:** Calculate $H$ if the policy  $\ell_{t}=\left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}}$ from question 1 is followed. Choose $K$ so the approximation is good enough to not affect your results substantially.

In [75]:
model = ProfitClass(do_print=False)
np.random.seed(21)
model.calculate_H()

Expected value of the salon (H): 31.067422809474735


In [77]:
model = ProfitClass(do_print=False)
model.Calculate_H_Delta()

AttributeError: 'ProfitClass' object has no attribute 'Calculate_H_Delta'

In [50]:
# Set parameter values
rho = 0.90
iota = 0.01
sigma_epsilon = 0.10
R = (1 + 0.01) ** (1/12)

# Specify the number of simulations
K = 1000

# Initialize variables
kappa_prev = 1.0
ell_prev = 0
ex_post_value = 0

# Simulation loop
for k in range(K):
    ex_post_value_k = 0
    kappa_t = np.zeros(120)
    
    for t in range(120):
        # Generate random shock
        epsilon_t = np.random.normal(-0.5 * sigma_epsilon**2, sigma_epsilon)
        
        # Compute demand shock for current month
        kappa_t[t] = np.exp(rho * np.log(kappa_prev) + epsilon_t)
        
        # Calculate optimal number of hairdressers
        ell_t = ((1 - 0.5) * kappa_t[t] / 1.0) ** (1 / 0.5)
        
        # Compute ex post value of the salon for current month
        ex_post_value_k += R**(-t) * (kappa_t[t] * ell_t**(1-0.5) - 1.0 * ell_t - (ell_t != ell_prev) * iota)
        
        # Update previous number of hairdressers
        ell_prev = ell_t
    
    # Add ex post value of the salon for current simulation to total ex post value
    ex_post_value += ex_post_value_k

# Calculate expected value of the salon
H = ex_post_value / K

# Print the result
print("Expected value of the salon (H):", H)

Expected value of the salon (H): 27.73285605953243


Next, we consider policies on the form:

$$

\ell_{t}=\begin{cases}
\ell_t^{\ast}  & \text{if }\left|\ell_{t-1}-\ell_t^{\ast} \right|>\Delta\\
\ell_{t-1} & \text{else }
\end{cases}
\\
\text{where}\,\,\ell_t^{\ast} = \left(\frac{(1-\eta)\kappa_{t}}{w}\right)^{\frac{1}{\eta}} \\

$$
With $\Delta \geq 0$ and $\Delta = 0$ being the previous policy.


**Question 3:** Calculate $H$ if the policy above was followed with $\Delta = 0.05$. Does it improve profitability?

In [59]:
# Specify the adjustment parameter
delta = 0.05

# Initialize variables
kappa_prev = 1.0
ell_prev = 0
ex_post_value = 0

# Simulation loop
for k in range(K):
    ex_post_value_k = 0
    kappa_t = np.zeros(120)
    
    for t in range(120):
        # Generate random shock
        epsilon_t = np.random.normal(-0.5 * sigma_epsilon**2, sigma_epsilon)
        
        # Compute demand shock for current month
        kappa_t[t] = np.exp(rho * np.log(kappa_prev) + epsilon_t)
        
        # Calculate optimal number of hairdressers
        ell_star = ((1 - 0.5) * kappa_t[t] / 1.0) ** (1 / 0.5)
        
        # Adjust number of hairdressers based on policy
        if abs(ell_prev - ell_star) > delta:
            ell_t = ell_star
        else:
            ell_t = ell_prev
        
        # Compute ex post value of the salon for current month
        ex_post_value_k += R**(-t) * (kappa_t[t] * ell_t**(1-0.5) - 1.0 * ell_t - (ell_t != ell_prev) * iota)
        
        # Update previous number of hairdressers
        ell_prev = ell_t
    
    # Add ex post value of the salon for current simulation to total ex post value
    ex_post_value += ex_post_value_k

# Calculate expected value of the salon
H = ex_post_value / K

# Print the result
print("Expected value of the salon (H):", H)

Expected value of the salon (H): 28.23760777155689


**Question 4:** Find the optimal $\Delta$ maximizing $H$. Illustrate your result.

In [60]:
# Specify the range of values for Delta
delta_values = np.linspace(0, 0.1, 11)  # Adjust the range as needed

# Initialize variables
optimal_delta = None
max_H = float('-inf')

# Perform grid search
for delta in delta_values:
    # Initialize variables
    kappa_prev = 1.0
    ell_prev = 0
    ex_post_value = 0
    
    # Simulation loop
    for k in range(K):
        ex_post_value_k = 0
        kappa_t = np.zeros(120)
        
        for t in range(120):
            # Generate random shock
            epsilon_t = np.random.normal(-0.5 * sigma_epsilon**2, sigma_epsilon)
            
            # Compute demand shock for current month
            kappa_t[t] = np.exp(rho * np.log(kappa_prev) + epsilon_t)
            
            # Calculate optimal number of hairdressers
            ell_star = ((1 - 0.5) * kappa_t[t] / 1.0) ** (1 / 0.5)
            
            # Adjust number of hairdressers based on policy
            if abs(ell_prev - ell_star) > delta:
                ell_t = ell_star
            else:
                ell_t = ell_prev
            
            # Compute ex post value of the salon for current month
            ex_post_value_k += R**(-t) * (kappa_t[t] * ell_t**(1-0.5) - 1.0 * ell_t - (ell_t != ell_prev) * iota)
            
            # Update previous number of hairdressers
            ell_prev = ell_t
        
        # Add ex post value of the salon for current simulation to total ex post value
        ex_post_value += ex_post_value_k
    
    # Calculate expected value of the salon
    H = ex_post_value / K
    
    # Update optimal delta if current H is higher
    if H > max_H:
        max_H = H
        optimal_delta = delta

# Print the result
print("Optimal Delta:", optimal_delta)
print("Maximum Expected value of the salon (H):", max_H)

Optimal Delta: 0.09
Maximum Expected value of the salon (H): 28.453200523499255



**Question 5:** Suggest an alternative policy you believe might improve profitability. Implement and test your policy.

In [ ]:
skriv svar her